In [1]:
!pip install wandb
import wandb
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets
from torch import optim
import torch.nn.functional as F
from torch.profiler import profile, record_function, ProfilerActivity
import re

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "/private/var/containers/Bundle/Application/1BBC9D97-DFE3-4B44-B797-60FBDDCD97B6/Carnets-sci.app/Library/lib/python3.11/site-packages/pip/_vendor/pep517/in_process/_in_process.py", line 351, in <module>
          main()
        File "/private/var/containers/Bundle/Application/1BBC9D97-DFE3-4B44-B797-60FBDDCD97B6/Carnets-sci.app/Library/lib/python3.11/site-packages/pip/_vendor/pep517/in_process/_in_process.py", line 333, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/private/var/containers/Bundle/Application/1BBC9D97-DFE3-4B44-B797-60FBDDCD97B6/Carnets-sci.app/Library/lib/python3.11/site-packages/pip/_vendor/pep517/in_process/_in_process.py", line 118, in get_requires_for_build_wheel

      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


ModuleNotFoundError: No module named 'wandb'

In [ ]:
def print_samples(trainset):
  # print a few samples
  figure = plt.figure(figsize=(6,6))
  cols, rows = 6, 6
  for i in range(1, cols*rows + 1):
    sample_idx = torch.randint(len(trainset), size=(1,)).item()
    img, label = trainset[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(label)
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap='gray')
  plt.tight_layout()
  plt.show()

In [ ]:
def build_dataset(num_classes):
  # take image, convert to (3, 28, 28) array with values in (0,255), then convert to (28,28) with values in (0,1) and normalize
  transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,)),])
  # initialize train, eval sets
  trainset = datasets.MNIST('../data', download=True, train=True, transform=transform)
  valset = datasets.MNIST('../data', download=True, train=False, transform=transform)

  train_idx = trainset.targets < num_classes
  trainset.data = trainset.data[train_idx]
  trainset.targets = trainset.targets[train_idx]
  test_idx = valset.targets < num_classes
  valset.data = valset.data[test_idx]
  valset.targets = valset.targets[test_idx]

  # initialize train, eval loaders
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
  valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

  # print_samples(trainset)
  
  return trainloader, valloader

In [ ]:
def train_epoch(model, train_loader, optimizer, robustness, batch_num):
  total_loss = 0
  for data, target in train_loader:
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()

    # if robust, perform FGSM attack with epsilon=robustness
    if robustness > 0:
      delta = torch.zeros_like(data, requires_grad=True)
      loss = F.cross_entropy(model(data + delta), target)
      loss.backward()
      data += robustness * delta.grad.detach().sign()

    # forward pass
    loss = F.cross_entropy(model(data), target)
    total_loss += loss.item()
    wandb.log({"batch_loss": loss.item(), "batch_num": batch_num})

    # backward pass
    loss.backward()
    optimizer.step()

    batch_num += 1

  return total_loss / len(train_loader), batch_num


@torch.no_grad()
def test_loss(model, test_loader):
  total_loss = 0
  total_acc = 0
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    pred = model(data)
    loss = F.cross_entropy(pred, target)
    total_loss += loss.item()
    total_acc += (torch.argmax(pred, dim=1) == target).sum().item()
  return (total_loss / len(test_loader)), (total_acc / len(test_loader.dataset))

In [ ]:
def extract_and_sum_flops(log_str):
    # Define a regex pattern to extract the units (KFLOPs, MFLOPs, GFLOPs) from the top line
    units_pattern = re.compile(r'Total\s(KFLOPs|MFLOPs|GFLOPs)')
    # Define a regex pattern to extract the FLOPs value from each line
    value_pattern = re.compile(r'\s+(\d+\.\d+)\s*$')
    
    total_mflops = 0.0
    units = None
    
    # Determine the units from the top line of the table
    for line in log_str.split('\n'):
        units_match = units_pattern.search(line)
        if units_match:
            units = units_match.group(1)
            break
    
    # Iterate over each line in the string to calculate the total FLOPs
    for line in log_str.split('\n'):
        value_match = value_pattern.search(line)
        if value_match:
            flops = float(value_match.group(1))
            # Convert to MFLOPs based on the units
            if units == 'KFLOPs':
                flops /= 1000.0  # Convert KFLOPs to MFLOPs
            elif units == 'GFLOPs':
                flops *= 1000.0  # Convert GFLOPs to MFLOPs
            total_mflops += flops

    return total_mflops

In [ ]:
def train(config=None):
  num_epochs = 20

  # initialize a new wandb run
  with wandb.init(config=config):
    # if this is called by wandb.agent, this config will be set by sweep controller
    config = wandb.config

    train_loader, test_loader = build_dataset(config.task_complexity)
    model = build_model(config.architecture, config.weight_class, config.task_complexity)
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=.001)
    epsilon = config.adv_robustness # will be evaluated as False if 0

    # train, log the train and test loss
    batch_num = 0
    for epoch in range(num_epochs):
      avg_loss, batch_num = train_epoch(model, train_loader, optimizer, epsilon, batch_num)
      avg_test_loss, avg_test_acc = test_loss(model, test_loader)
      wandb.log({"train_loss": avg_loss, "test_loss": avg_test_loss, "test_acc": avg_test_acc, "epoch": epoch})
      if epoch == 9:
        optimizer = optim.SGD(model.parameters(), lr=.0001)
    wandb.log({"loss": avg_test_loss})

    # log cpu time and MFLOPs for 20 samples
    inputs = torch.rand(20, 1, 28, 28)
    with profile(activities=[ProfilerActivity.CPU],
        with_flops=True) as prof:
          model(inputs)
    wandb.log({"cpu_time_ms": prof.key_averages().self_cpu_time_total, 
               'inference_MFLOPs': extract_and_sum_flops(str(prof.key_averages()))})
    wandb.alert(title='Run Complete', text=f"loss: {avg_test_loss:.3f}, type (W-A-C-R): {config.weight_class}-{config.architecture}-{config.task_complexity}-{config.adv_robustness}")

In [ ]:
sweep_configuration = {
    'method': 'random',
    'name': 'new_CNN_models',
    'metric': {
        'goal': 'minimize',
        'name': 'loss'
    },
    'parameters': {
        # 'architecture': {'values': ['MLP', 'CNN', 'MIX']},
        'architecture': {'value': 'MLP'},
        'adv_robustness': {'values': [0, .1, 1]},
        # 'adv_robustness': {'value': .1},
        'weight_class': {'values': ['S', 'M', 'L']},
        # 'weight_class': {'value': 'S'},
        'task_complexity': {'values': [2, 5, 10]}
        # 'task_complexity': {'value': 2}
    }
}
# sweep_id = wandb.sweep(sweep_configuration, project='potential-energy')
wandb.agent(sweep_id='jvorlm29', function=train, project="potential-energy")
# wandb.agent(sweep_id, train)

In [ ]:
# class Config:
#   def __init__(self):
#     self.task_complexity = 5
#     self.architecture = 'MLP'
#     self.weight_class = 'S'
#     self.adv_robustness = 0

# config = Config()
# train(config)

# loader = build_dataset(2)
# model = build_model('MLP', 'S', 2)
# optimizer = optim.SGD(model.parameters(), lr=.001)
# epsilon = 0 # will be evaluated as False if 0

# train_epoch(model, loader, optimizer, epsilon)